In [130]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
from IPython.display import display

titles = []
written_times = []
first_lines = []

for i in range(31,33):
    url = "https://www.reuters.com/news/archive/GCA-ForeignExchange?view=page&page={}&pageSize=10".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    for title in soup.find_all('h3', {'class':'story-title'}):
        titles.append(title.string.strip())
        
    for written_time in soup.find_all('span', {'class':'timestamp'}):
        written_times.append(written_time.string.strip())
        
        
    try:
        for first_line in soup.find_all('p'):
            first_lines.append(first_line.string.strip())
    except:
        pass
    
    titles = titles[:len(titles)-3]
    first_lines = first_lines[:len(first_lines)-2]
        
reuters_word_headlines = pd.DataFrame({'title':titles, 'time':written_times, 'headline content':first_lines})
reuters_word_headlines = reuters_word_headlines.loc[:,('time', 'title','headline content')]

reuters_word_headlines.to_csv(path_or_buf='/home/tushar/Desktop/Python/Python3/tradegame-master/output.csv',index=False)

In [205]:
#df = pd.DataFrame
df = pd.read_csv('/home/tushar/Desktop/Python/Python3/tradegame-master/output.csv')
df = df.loc[9:]
df = df.loc[:9824]
df.reset_index(drop=True, inplace=True)

In [266]:
df_new = df.copy(deep=True)
df_new['compound']=''
df_new['neg']=''
df_new['neu']=''
df_new['pos']=''

In [203]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tushar/nltk_data...


True

In [264]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()

In [267]:
for index,sentence in enumerate(df_new['headline content']):
    score = sia.polarity_scores(sentence)
    df_new['compound'][index]=score['compound']
    df_new['neg'][index]=score['neg']
    df_new['neu'][index]=score['neu']
    df_new['pos'][index]=score['pos']

In [421]:
searchfor = ['pound','EU','sterling','UK']
df_eu = df_new[df_new['title'].str.contains("|".join(searchfor), regex=True, case=False, na=False)].copy(deep=True)

In [422]:
df_eu['time'] = pd.to_datetime(df['time'])
#df_eu = df_eu.drop(['title','headline content'],axis = 1)

In [423]:
df_eu['compound'] = df_eu['compound'].astype(float)
df_eu['pos'] = df_eu['pos'].astype(float)
df_eu['neu'] = df_eu['neu'].astype(float)
df_eu['neg'] = df_eu['neg'].astype(float)

In [425]:
gp = df_eu.groupby('time',as_index=False)['compound','neg','neu','pos'].mean()
gp = gp.set_index('time')

In [366]:
gp

,compound,neg,neu,pos
time,,,,
2012-01-06,0.381800,0.000000,0.934000,0.066000
2012-01-11,-0.585900,0.226000,0.652000,0.122000
2012-01-12,-0.307100,0.095000,0.905000,0.000000
2012-01-24,0.261700,0.000000,0.927000,0.073000
2012-01-27,0.214400,0.147000,0.672000,0.181000
2012-01-31,0.214400,0.147000,0.672000,0.181000
2012-02-01,0.873800,0.000000,0.726000,0.274000
2012-02-08,0.510600,0.000000,0.916000,0.084000
2012-02-16,0.361200,0.000000,0.923000,0.077000


In [426]:
comb = pd.date_range(start='1/1/2012', end='31/12/2018')
new = pd.DataFrame(comb)
new['compound'] = 0
new['pos'] = 0
new['neu'] = 1
new['neg'] = 0
new['compound'] = new['compound'].astype(float)
new['pos'] = new['pos'].astype(float)
new['neu'] = new['neu'].astype(float)
new['neg'] = new['neg'].astype(float)
new = new.set_index(0)

In [427]:
new.loc[gp.index,'compound'] = gp['compound']
new.loc[gp.index,'pos'] = gp['pos']
new.loc[gp.index,'neu'] = gp['neu']
new.loc[gp.index,'neg'] = gp['neg']

In [430]:
new = new[1:]

In [431]:
upsampled = new.resample('15T').ffill()

In [438]:
upsampled = upsampled[:-1]

In [439]:
upsampled

,compound,pos,neu,neg
0,,,,
2012-01-02 00:00:00,0.0,0.0,1.0,0.0
2012-01-02 00:15:00,0.0,0.0,1.0,0.0
2012-01-02 00:30:00,0.0,0.0,1.0,0.0
2012-01-02 00:45:00,0.0,0.0,1.0,0.0
2012-01-02 01:00:00,0.0,0.0,1.0,0.0
2012-01-02 01:15:00,0.0,0.0,1.0,0.0
2012-01-02 01:30:00,0.0,0.0,1.0,0.0
2012-01-02 01:45:00,0.0,0.0,1.0,0.0
2012-01-02 02:00:00,0.0,0.0,1.0,0.0


In [406]:
df_2012 = pd.read_csv('/home/tushar/Desktop/Python/Python3/tradegame-master/data/sampled_data_15T/yearly/EURUSD_2012.csv')

In [408]:
df_2012 = df_2012.set_index('tick')

In [464]:
df_2012 = pd.read_csv('/home/tushar/Desktop/Python/Python3/tradegame-master/data/sampled_data_15T/yearly/EURUSD_2017.csv')
df_2012['tick'] = pd.to_datetime(df_2012['tick'])
df_2012 = df_2012.set_index('tick')
df_2012['compound'] = 0
df_2012['pos'] = 0
df_2012['neu'] = 1
df_2012['neg'] = 0
df_2012['compound'] = df_2012['compound'].astype(float)
df_2012['pos'] = df_2012['pos'].astype(float)
df_2012['neu'] = df_2012['neu'].astype(float)
df_2012['neg'] = df_2012['neg'].astype(float)
idx1 = upsampled.index
idx2 = df_2012.index
idx = idx1.intersection(idx2)
df_2012.loc[idx,'compound'] = upsampled['compound']
df_2012.loc[idx,'pos'] = upsampled['pos']
df_2012.loc[idx,'neu'] = upsampled['neu']
df_2012.loc[idx,'neg'] = upsampled['neg']
df_2012.to_csv(path_or_buf='/home/tushar/Desktop/Python/Python3/tradegame-master/data/EURUSD_2017_new.csv')
#df_2012.loc[upsampled.index,'compound'] = upsampled['compound']
#df_2012.loc[upsampled.index,'pos'] = upsampled['pos']
#df_2012.loc[upsampled.index,'neu'] = upsampled['neu']
#df_2012.loc[upsampled.index,'neg'] = upsampled['neg']